## OCR任务说明
<font size=3> OCR任务分为两种：文字检测和文字识别，检测需要定位文本位置(类似目标检测)，而识别就是识别出图像中的文字，我们这次的比赛任务就是文字识别。
 
![](https://ai-studio-static-online.cdn.bcebos.com/85a90ae3766144b6bf35393c831e44b99e05c0fb348944bdb73957c804ce60ab)
    

## 基线选择
<font size=3> 对于OCR任务，我们首推PaddleOCR工具包，PaddleOCR中集成了非常多的OCR算法可供大家选择！如果你想更加深入地了解PaddleOCR，可以去github上查看，链接在这:[PaddleOCR](https://github.com/PaddlePaddle/PaddleOCR)。

### 0、安装必要的依赖库
<font size=3> 我们首先需要安装一些机器学习、深度学习、图像处理的Python库。

In [1]:
!pip install -r PaddleOCR/requirements.txt

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 1.4 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.6 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━

### 1、数据集准备
<font size=3> 我们需要解压数据，这里需要注意地是aistudio中数据最好放在data目录下，因为data目录下空间很大，放在其他地方会导致项目加载变慢。

In [2]:
#解压数据集
#!unzip -d data/ data/data258841/DataForCompetitor.zip
!unzip -d data/ data/hand_written.zip

  inflating: data/hand_written/91.jpg  
  inflating: data/hand_written/910.jpg  
  inflating: data/hand_written/9100.jpg  
  inflating: data/hand_written/9101.jpg  
  inflating: data/hand_written/9102.jpg  
  inflating: data/hand_written/9103.jpg  
  inflating: data/hand_written/9104.jpg  
  inflating: data/hand_written/9105.jpg  
  inflating: data/hand_written/9106.jpg  
  inflating: data/hand_written/9107.jpg  
  inflating: data/hand_written/9108.jpg  
  inflating: data/hand_written/9109.jpg  
  inflating: data/hand_written/911.jpg  
  inflating: data/hand_written/9110.jpg  
  inflating: data/hand_written/9111.jpg  
  inflating: data/hand_written/9112.jpg  
  inflating: data/hand_written/9113.jpg  
  inflating: data/hand_written/9114.jpg  
  inflating: data/hand_written/9115.jpg  
  inflating: data/hand_written/9116.jpg  
  inflating: data/hand_written/9117.jpg  
  inflating: data/hand_written/9118.jpg  
  inflating: data/hand_written/9119.jpg  
  inflating: data/hand_written/912.jpg

<font size=3> 我们先对数据集的基本情况看一下，先来看看图片

![](https://ai-studio-static-online.cdn.bcebos.com/6f5b7c81146e442f85a4479e739099aa3308beec814748e19484b868922f1e87)
    
![](https://ai-studio-static-online.cdn.bcebos.com/51595b38ff7c4d7a9ddecee0023060271736fa5ee0b44fcb93244c015a1be6b4)
    
![](https://ai-studio-static-online.cdn.bcebos.com/d8f943544aa24d0ea4e3e5a4883ce105e6f82a25433b4d458d492e9ffe288d59)

<font size=3> 下面是标签
    
![](https://ai-studio-static-online.cdn.bcebos.com/fd185d7e8a6f4920bb0e5087a6f2f7c30fdc3cb4474f4a44a0d32f40f1aecc81)
    
    
<font size=3> 从标签的情况来看，数据集包含了英文、中文、字符等，本项目中我们使用了PaddleOCR自带的字典进行训练，当然大家可以根据数据集情况来定制化字典。
并且，数据集质量不是很高，图片分辨率很低很低，这就是一个需要解决的问题，大家可以开动脑筋。

<font size=3> **数据集的一些Tips**: (1) 大家要清楚地了解数据集的基本情况，包括图片质量，数据的分布。历年来看，最后高分选手都是对数据集进行了仔细地清洗。(2) 合成数据尽量少用，因为这个会导致结果无法重现。

In [3]:
#数据集预处理（二值化）
from PIL import Image
import os

# 设置输入和输出文件夹路径
input_folder = 'data/DataForCompetitor/train'  # 输入文件夹路径
output_folder = 'data/DataForCompetitor/train_preprocess'  # 输出文件夹路径

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 获取输入文件夹中的所有图片文件
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]

# 处理前十张图片
for i, image_file in enumerate(image_files[:10]):
    # 加载图片
    image_path = os.path.join(input_folder, image_file)
    image = Image.open(image_path)
    
    # 将图片转换为灰度图
    gray_image = image.convert('L')
    
    # 将灰度图二值化，你可以根据需要调整阈值（这里设置为128）
    binary_image = gray_image.point(lambda x: 255 if x > 128 else 0, '1')
    
    # 保存处理后的图片
    output_path = os.path.join(output_folder, f'binary_{image_file}')
    binary_image.save(output_path)
    
    print(f'Processed image {i+1}: {image_file} -> {output_path}')

Processed image 1: mg_crop_249089.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_249089.jpg
Processed image 2: mg_crop_171146.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_171146.jpg
Processed image 3: mg_crop_156779.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_156779.jpg
Processed image 4: mg_crop_230763.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_230763.jpg
Processed image 5: mg_crop_159910.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_159910.jpg
Processed image 6: mg_crop_214908.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_214908.jpg
Processed image 7: mg_crop_44437.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_44437.jpg
Processed image 8: mg_crop_247342.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_247342.jpg
Processed image 9: mg_crop_284296.jpg -> data/DataForCompetitor/train_preprocess/binary_mg_crop_284296.jpg
Processed image 10: mg_crop_231042.jpg 

### 2、克隆项目仓库
<font size=3> 接下来我们克隆对应的项目仓库，注意，本项目一定要用PaddleOCR2.7才可以完成。另外，很多同学最后肯定会遇到文件过大问题，本项目对PaddleOCR进行了清洗删除了不必要部分，大家可以直接使用！

In [2]:
#PPOCRv4目前只在github上面，注意不要clone错了
#目前项目里面已经上传最新的PaddleOCR
# !git clone https://github.com/PaddlePaddle/PaddleOCR

### 2、开始炼丹
<font size=3> 准备好代码仓库和数据，那么下面就是快乐炼丹了。

In [3]:
# 解压一下预训练权重
!tar -xvf ch_PP-OCRv4_rec_train.tar

ch_PP-OCRv4_rec_train/
ch_PP-OCRv4_rec_train/._student.pdparams
ch_PP-OCRv4_rec_train/student.pdparams


In [4]:
#进入PaddleOCR目录
%cd /home/aistudio/PaddleOCR

/home/aistudio/PaddleOCR


In [5]:
#开始执行训练
!python tools/train.py -c configs/rec/PP-OCRv4/ch_PP-OCRv4_rec.yml -o Global.pretrained_model=/home/aistudio/ch_PP-OCRv4_rec_train/student.pdparams

[2024/12/03 14:54:32] ppocr INFO: Architecture : 
[2024/12/03 14:54:32] ppocr INFO:     Backbone : 
[2024/12/03 14:54:32] ppocr INFO:         name : PPLCNetV3
[2024/12/03 14:54:32] ppocr INFO:         scale : 0.95
[2024/12/03 14:54:32] ppocr INFO:     Head : 
[2024/12/03 14:54:32] ppocr INFO:         head_list : 
[2024/12/03 14:54:32] ppocr INFO:             CTCHead : 
[2024/12/03 14:54:32] ppocr INFO:                 Head : 
[2024/12/03 14:54:32] ppocr INFO:                     fc_decay : 1e-05
[2024/12/03 14:54:32] ppocr INFO:                 Neck : 
[2024/12/03 14:54:32] ppocr INFO:                     depth : 2
[2024/12/03 14:54:32] ppocr INFO:                     dims : 120
[2024/12/03 14:54:32] ppocr INFO:                     hidden_dims : 120
[2024/12/03 14:54:32] ppocr INFO:                     kernel_size : [1, 3]
[2024/12/03 14:54:32] ppocr INFO:                     name : svtr
[2024/12/03 14:54:32] ppocr INFO:                     use_guide : True
[2024/12/03 14:54:32] ppocr 

<font size=3> 这里我们选择的是最新的PPOCRv4，这里向大家介绍一下配置文件内容。


```
Global:
  debug: false
  use_gpu: true #是否使用GPU训练
  epoch_num: 50 #训练轮数
  log_smooth_window: 20 #常规log设置
  print_batch_step: 10 #打印训练日志的迭代次数
  save_model_dir: ./output/rec_ppocr_v4 #模型保存路径
  save_epoch_step: 10 #保存模型的轮数
  eval_batch_step: [0, 2000] #进行evaluation的迭代次数
  cal_metric_during_train: true #训练过程中是否计算指标
  pretrained_model: #预训练模型权重
  checkpoints: #恢复训练的权重
  save_inference_dir: #保存推理结果的路径
  use_visualdl: false #是否可视化训练过程
  infer_img: doc/imgs_words/ch/word_1.jpg #推理时候的图片
  character_dict_path: ppocr/utils/ppocr_keys_v1.txt
  max_text_length: &max_text_length 25
  infer_mode: false
  use_space_char: true
  distributed: true #是否分布式训练
  save_res_path: ./output/rec/predicts_ppocrv3.txt #保存预测结果的txt路径


Optimizer: #优化器参数
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Cosine
    learning_rate: 0.0001
    warmup_epoch: 5
  regularizer:
    name: L2
    factor: 3.0e-05


Architecture: #模型结构
  model_type: rec #文本识别
  algorithm: SVTR_LCNet #算法名称
  Transform:
  Backbone: #骨干网络
    name: PPLCNetV3
    scale: 0.95
  Head: #预测头
    name: MultiHead
    head_list:
      - CTCHead:
          Neck:
            name: svtr
            dims: 120
            depth: 2
            hidden_dims: 120
            kernel_size: [1, 3]
            use_guide: True
          Head:
            fc_decay: 0.00001
      - NRTRHead:
          nrtr_dim: 384
          max_text_length: *max_text_length

Loss: #损失函数
  name: MultiLoss
  loss_config_list:
    - CTCLoss:
    - NRTRLoss:

PostProcess:  #后处理
  name: CTCLabelDecode

Metric: #验证时候的指标
  name: RecMetric
  main_indicator: acc

Train: #训练过程配置
  dataset: #数据集信息
    name: MultiScaleDataSet
    ds_width: false
    data_dir: /home/aistudio/data/DataForCompetitor/ #数据集根路径
    ext_op_transform_idx: 1
    label_file_list: #标签路径
    - /home/aistudio/data/DataForCompetitor/train_label.txt
    transforms: #数据增强
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - RecConAug:
        prob: 0.5
        ext_data_num: 2
        image_shape: [48, 320, 3]
        max_text_length: *max_text_length
    - RecAug:
    - MultiLabelEncode:
        gtc_encode: NRTRLabelEncode
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_gtc
        - length
        - valid_ratio
  sampler: 
    name: MultiScaleSampler
    scales: [[320, 32], [320, 48], [320, 64]]
    first_bs: &bs 192
    fix_bs: false
    divided_factor: [8, 16] # w, h
    is_training: True
  loader:
    shuffle: true
    batch_size_per_card: *bs
    drop_last: true
    num_workers: 8
Eval: #验证信息
  dataset:
    name: SimpleDataSet
    data_dir: /home/aistudio/data/DataForCompetitor/
    label_file_list:
    - /home/aistudio/data/DataForCompetitor/train_label.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - MultiLabelEncode:
        gtc_encode: NRTRLabelEncode
    - RecResizeImg:
        image_shape: [3, 48, 320]
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_gtc
        - length
        - valid_ratio
  loader:
    shuffle: false
    drop_last: false
    batch_size_per_card: 128
    num_workers: 4

```


### 3、导出模型
<font size=3> 我们需要将模型导出，以去除掉不必要的参数。

In [6]:
#注意修改下面的路径
# -c 后面设置训练算法的yml配置文件
# -o 配置可选参数
# Global.pretrained_model 参数设置待转换的训练模型地址，不用添加文件后缀 .pdmodel，.pdopt或.pdparams。
# Global.save_inference_dir参数设置转换的模型将保存的地址
!python tools/export_model.py -c configs/rec/PP-OCRv4/ch_PP-OCRv4_rec.yml -o Global.pretrained_model=/home/aistudio/output/rec_ppocr_v4/best_model/model  Global.save_inference_dir=/home/aistudio/infer/

W1203 16:52:44.371107 124230 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
W1203 16:52:44.372476 124230 gpu_resources.cc:164] device: 0, cuDNN Version: 8.9.
[2024/12/03 16:52:47] ppocr INFO: load pretrain successful from /home/aistudio/output/rec_ppocr_v4/best_model/model
I1203 16:52:53.726621 124230 program_interpreter.cc:212] New Executor is Running.
[2024/12/03 16:52:53] ppocr INFO: inference model is saved to /home/aistudio/infer/inference


In [1]:
!zip -r rec_infer.zip infer

  adding: infer/ (stored 0%)
  adding: infer/inference.pdmodel (deflated 90%)
  adding: infer/inference.pdiparams (deflated 9%)
  adding: infer/inference.pdiparams.info (deflated 82%)


### 4、准备提交结果
<font size=3> 最后一步就是准备预测结果就行啦

In [7]:
%cd /home/aistudio/

/home/aistudio


In [8]:
# 调试一下predict.py是否正确
!python predict.py

[2024/12/03 16:54:00] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320


In [9]:
#打包, 下载, 提交
!zip -q -r -o submission.zip model/ PaddleOCR/ predict.py

<font size=3> 压缩包的结构如下，**提交结果一定要将PaddleOCR目录下训练好的权重目录删除**，这些会导致目录超100M。这些模型都用不到，我们用导出模型预测。

![](https://ai-studio-static-online.cdn.bcebos.com/353b13b9a0a64319971b52edcebb427c8884aee84bac40e7ab1cfeb3db940c89)

<font size=3> 提交的PaddleOCR的目录如下
    
 ![](https://ai-studio-static-online.cdn.bcebos.com/a779dbb056e24b318f574138d418a0385bb2eb5768ae489391d1a2e72f2f6eec)
    

### 5、评价指标

<font size=3>  **Accuracy** : 模型对每张图片里文字内容的识别准确率，错一个字即为错，按照准确率高低进行排名。

## 总结
    
<font size=3>本项目中我们选择的模型是轻量化模型，大家可以尝试更大的模型，另外此次线上赛不限制推理速度，那么可以看看Transformer模型。